In [1]:
import pandas as pd
import numpy as np
import pickle
import uuid
from datetime import datetime

In [2]:
CATEGORY_MAPPING = {
    1: "Electronics",
    2: "Apparel",
    3: "Books",
    4: "Home Goods"
}

class CustomerDataExtractor:
    def __init__(self, pickle_path, vip_txt_path=None):
        self.pickle_path = pickle_path
        self.vip_ids = set()
        if vip_txt_path:
            with open(vip_txt_path, "r") as f:
                self.vip_ids = set(map(int, f.read().splitlines()))

    def _parse_price(self, val):
        if isinstance(val, str):
            val = val.strip().replace("$", "")
            if not val.replace(".", "", 1).isdigit():
                return np.nan
        try:
            return float(val)
        except:
            return np.nan

    def _parse_quantity(self, val):
        if isinstance(val, str):
            val = val.strip()
            if val.upper() == "FREE":
                return 0
            if not val.isdigit():
                return np.nan
        try:
            return int(val)
        except:
            return np.nan

    def _normalize_category(self, raw_cat):
        if isinstance(raw_cat, int):
            return CATEGORY_MAPPING.get(raw_cat, "Misc")
        elif isinstance(raw_cat, str):
            raw = raw_cat.strip().lower()
            for v in CATEGORY_MAPPING.values():
                if v.lower() == raw:
                    return v
        return "Misc"

    def _safe_date(self, date_val):
        try:
            return pd.to_datetime(date_val, errors='coerce')
        except:
            return pd.NaT

    def _normalize_order_id(self, raw_id):
        try:
            if raw_id is None:
                return -1
            if isinstance(raw_id, str):
                raw_id = raw_id.strip().replace("ORD", "")
            return int(raw_id)
        except:
            return -1

    def transform(self):
        with open(self.pickle_path, "rb") as f:
            data = pickle.load(f)

        records = []

        for cust in data:
            customer_id = cust.get("id")
            customer_name = cust.get("name", "Unknown")
            reg_date = self._safe_date(cust.get("registration_date"))
            is_vip = customer_id in self.vip_ids

            for order in cust.get("orders", []):
                order_id = self._normalize_order_id(order.get("order_id"))
                order_date = self._safe_date(order.get("order_date"))
                order_total = order.get("order_total_value", 0.0) or 0.0
                items = order.get("items", [])
                if not items:
                    continue  # Skip orders with no items

                for item in items:
                    product_id = item.get("item_id")
                    if product_id is None:
                        continue
                    product_name = item.get("product_name", "").strip() or "Unknown"
                    category = self._normalize_category(item.get("category"))
                    price = self._parse_price(item.get("price"))
                    qty = self._parse_quantity(item.get("quantity"))

                    if np.isnan(price) or np.isnan(qty) or qty == 0:
                        continue

                    total_price = price * qty
                    pct = (total_price / order_total) * 100 if order_total else 0.0

                    # Add the record
                    records.append({
                        "customer_id": customer_id,
                        "customer_name": customer_name,
                        "registration_date": reg_date,
                        "is_vip": is_vip,
                        "order_id": order_id,
                        "order_date": order_date,
                        "product_id": product_id,
                        "product_name": product_name,
                        "category": category,
                        "unit_price": price,
                        "item_quantity": qty,
                        "total_item_price": total_price,
                        "total_order_value_percentage": pct
                    })

        df = pd.DataFrame(records)

        # Recalculate percentage using cleaned totals per order
        order_totals = df.groupby(['customer_id', 'order_id'])['total_item_price'].transform('sum')
        df['total_order_value_percentage'] = (df['total_item_price'] / order_totals) * 100.0

        # Drop zero-quantity items
        df = df[df['item_quantity'] > 0]

        # Clip invalid percentage values
        df['total_order_value_percentage'] = df['total_order_value_percentage'].clip(0, 100)

        # Ensure data types
        df = df.astype({
            "customer_id": "int64",
            "customer_name": "string",
            "is_vip": "bool",
            "order_id": "int64",
            "product_id": "int64",
            "product_name": "string",
            "category": "string",
            "unit_price": "float64",
            "item_quantity": "int64",
            "total_item_price": "float64",
            "total_order_value_percentage": "float64"
        })
        df["registration_date"] = pd.to_datetime(df["registration_date"], errors="coerce", utc=True)
        df["order_date"] = pd.to_datetime(df["order_date"], errors="coerce", utc=True)

        # Final sort
        df = df.sort_values(by=["customer_id", "order_id", "product_id"]).reset_index(drop=True)
        return df


In [3]:
extractor = CustomerDataExtractor("customer_orders.pkl", "vip_customers.txt")
df = extractor.transform()
df.to_csv("final_output6.csv", index=False)